In [1]:
import os
import csv
import subprocess
import pandas as pd

def get_video_metadata(file_path):
    """Extract metadata from a video file using ffprobe."""
    file_size = os.path.getsize(file_path)  # Get file size in bytes
    file_size_mb = round(file_size / (1024 * 1024), 2)  # Convert to MB
    
    cmd = [
        "ffprobe", "-v", "error",
        "-select_streams", "v:0",
        "-show_entries", "stream=width,height,r_frame_rate,bit_rate",
        "-show_entries", "format=duration",
        "-of", "csv=p=0",
        file_path
    ]
    
    try:
        output = subprocess.check_output(cmd).decode("utf-8").strip()
        width, height, frame_rate, bit_rate, duration = output.split(',')
        
        # Convert frame rate to a float value
        num, den = frame_rate.split('/')
        fps = round(float(num) / float(den), 2)
        
        # Calculate total frame count
        total_frames = int(float(duration) * fps)
        
        return [os.path.basename(file_path), file_size_mb, width, height, fps, bit_rate, duration, total_frames]
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return [os.path.basename(file_path), file_size_mb, "Error", "Error", "Error", "Error", "Error", "Error"]

def process_videos_in_folder(folder_path):
    """Process all DAV files in a folder and return metadata as a DataFrame."""
    video_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".dav")]
    
    data = []
    for video in video_files:
        video_path = os.path.join(folder_path, video)
        metadata = get_video_metadata(video_path)
        data.append(metadata)
    
    df = pd.DataFrame(data, columns=["Filename", "File Size (MB)", "Width", "Height", "FPS", "Bit Rate", "Duration (s)", "Total Frames"])
    return df

# Set your folder path containing DAV files
folder_path = r"C:\Users\humkabir\Desktop\CVAT\Smart Camera"  # Change this to your actual path

# Process videos and display results
df_metadata = process_videos_in_folder(folder_path)
display(df_metadata)  # Show table in Jupyter notebook

# Save to CSV
df_metadata.to_csv("video_metadata.csv", index=False)
print("Metadata extraction completed. Results saved to video_metadata.csv")


Error processing C:\Users\humkabir\Desktop\CVAT\Smart Camera\14.28.44-14.29.11[M][0@0][0].dav: [WinError 2] The system cannot find the file specified


,Filename,File Size (MB),Width,Height,FPS,Bit Rate,Duration (s),Total Frames
0,14.28.44-14.29.11[M][0@0][0].dav,33.84,Error,Error,Error,Error,Error,Error


PermissionError: [Errno 13] Permission denied: 'video_metadata.csv'